### Run the following cell only once after kernel started

In [1]:
import os
os.chdir('../')

### Make sure you are in "ondewo-t2s-client-python" folder

In [2]:
os.getcwd()

'/home/fcavallin/ondewo/ondewo-t2s'

In [3]:
import io
import soundfile as sf
import IPython.display as ipd
import grpc
from ondewo_grpc.ondewo.t2s import text_to_speech_pb2, text_to_speech_pb2_grpc
import google.protobuf.empty_pb2 as empty_pb2
from google.protobuf.json_format import ParseDict, MessageToDict, MessageToJson

### Set up the parameters of the grpc server. The example below is for the case when server is running locally

In [4]:
MAX_MESSAGE_LENGTH: int = 60000000
GRPC_HOST: str = "localhost"
GRPC_PORT: str = "50555"
CHANNEL: str = f"{GRPC_HOST}:{GRPC_PORT}"

options = [
    ('grpc.max_send_message_length', MAX_MESSAGE_LENGTH),
    ('grpc.max_receive_message_length', MAX_MESSAGE_LENGTH),
]

channel = grpc.insecure_channel(CHANNEL, options=options)

stab = text_to_speech_pb2_grpc.Text2SpeechStub(channel=channel)


### List all t2s pipelines present on the server

In [5]:
pipelines = stab.ListT2sPipelines(request=empty_pb2.Empty()).pipelines

### Select pipelines for specific language language

In [6]:
def find_pipeline_for_language(pipelines, language):
    for pipeline in pipelines:
        if pipeline.description.language == language:
            return pipeline

In [7]:
english_pipeline = find_pipeline_for_language(pipelines=pipelines, language='en')
german_pipeline = find_pipeline_for_language(pipelines=pipelines, language='de')

In [8]:
pipelines

[id: "glow_tts&hifi_gan-e976dd6c-2f41-484b-aec2-3e6868d37290"
description {
  language: "de"
  speaker_sex: "female"
  pipeline_owner: "ondewo"
  comments: "trained on public domain dataset"
  speaker_name: "Kerstin"
  domain: "general"
}
active: true
inference {
  type: "composite"
  composite_inference {
    text2mel {
      type: "glow_tts"
      glow_tts {
        batch_size: 5
        length_scale: 1.0
        noise_scale: 0.6669999957084656
        path: "models/glow-tts/de/kerstin_blank.pth"
        param_config_path: "models/glow-tts/de/config_blank.json"
      }
      glow_tts_triton {
        batch_size: 8
        length_scale: 1.0
        noise_scale: 0.6669999957084656
        max_text_length: 100
        param_config_path: "models/glow-tts/de/config_blank.json"
        triton_url: "localhost:50511"
        triton_model_name: "glow_tts"
      }
    }
    mel2audio {
      type: "hifi_gan"
      mb_melgan_triton {
        config_path: "models/mb_melgan/en/config.yml"
       

### Make synthesize request to the server to get audio for given text

In [9]:
config = text_to_speech_pb2.RequestConfig(t2s_pipeline_id=german_pipeline.id, # hgjhg jh
                                          length_scale=1.0, 
                                          pcm=0, 
                                          audio_format=text_to_speech_pb2.AudioFormat.wav)
request = text_to_speech_pb2.SynthesizeRequest(text='magic_word', 
                                               config=config)
response = stab.Synthesize(request=request)

In [10]:
response

audio_uuid: "81540761-c025-4786-8766-6492c3d85978"
audio: "RIFF$\026\001\000WAVEfmt \020\000\000\000\001\000\001\000\"V\000\000D\254\000\000\002\000\020\000data\000\026\001\000\007\000\017\000\013\000\n\000\022\000\017\000\024\000\017\000\022\000\022\000\021\000\021\000\034\000\033\000\031\000\025\000\025\000\022\000\020\000\016\000\023\000\024\000\022\000\017\000\017\000\r\000\016\000\n\000\014\000\016\000\n\000\t\000\020\000\013\000\010\000\004\000\005\000\014\000\014\000\010\000\t\000\006\000\014\000\010\000\005\000\000\000\377\377\002\000\007\000\007\000\006\000\007\000\n\000\004\000\004\000\003\000\002\000\005\000\006\000\n\000\016\000\017\000\014\000\003\000\005\000\004\000\010\000\013\000\014\000\r\000\014\000\n\000\005\000\375\377\004\000\002\000\005\000\014\000\r\000\t\000\013\000\r\000\004\000\002\000\000\000\003\000\010\000\t\000\016\000\020\000\n\000\000\000\375\377\000\000\002\000\003\000\002\000\007\000\n\000\002\000\377\377\375\377\365\377\355\377\366\377\363\377\370\377

In [11]:
config = text_to_speech_pb2.RequestConfig(t2s_pipeline_id=german_pipeline.id, length_scale = 1.0, pcm=0, audio_format= 0)
request = text_to_speech_pb2.SynthesizeRequest(text='magic_word', config=config)
response = stab.Synthesize(request=request)
print(f'Length of the generated audio is {response.audio_length} sec.', f'Generation time is {response.generation_time} sec.')

bio = io.BytesIO(response.audio)

audio = sf.read(bio, )

ipd.Audio(audio[0], rate=audio[1])

Length of the generated audio is 1.6137868165969849 sec. Generation time is 0.7884728312492371 sec.


In [12]:
config = text_to_speech_pb2.RequestConfig(t2s_pipeline_id=german_pipeline.id, length_scale = 1.0)
request = text_to_speech_pb2.SynthesizeRequest(text='Masada ist eine archäologische Stätte in Israel. Auf einem Gipfelplateau am Rand der Judäischen Wüste', config=config)
response = stab.Synthesize(request=request)

print(f'Length of the generated audio is {response.audio_length} sec.', f'Generation time is {response.generation_time} sec.')

bio = io.BytesIO(response.audio)

audio = sf.read(bio)

ipd.Audio(audio[0], rate=audio[1])

Length of the generated audio is 9.073559761047363 sec. Generation time is 2.921107769012451 sec.


### Adding length scale parameter to make speech faster or slower

In [13]:
config = text_to_speech_pb2.RequestConfig(t2s_pipeline_id=english_pipeline.id)
request = text_to_speech_pb2.SynthesizeRequest(text='Hi, how are you?', config=config)
response = stab.Synthesize(request=request)

print(f'Length of the generated audio is {response.audio_length} sec.', f'Generation time is {response.generation_time} sec.')

bio = io.BytesIO(response.audio)

audio = sf.read(bio)

ipd.Audio(audio[0], rate=audio[1])

Length of the generated audio is 1.05650794506073 sec. Generation time is 0.4851219058036804 sec.


### Get pipeline you want to update

In [14]:
request = text_to_speech_pb2.T2sPipelineId(id=english_pipeline.id)

In [15]:
request = text_to_speech_pb2.T2sPipelineId(id=english_pipeline.id)
pipeline_config = stab.GetT2sPipeline(request=request)

### Change parameter in the pipeline config. For example default length scale

In [16]:
pipeline_config.inference.composite_inference.text2mel.glow_tts.length_scale = 2

In [17]:
pipeline_config

id: "glow_tts&hifi_gan-e976dd6c-2f41-484b-aec2-3e6868d37280"
description {
  language: "en"
  speaker_sex: "female"
  pipeline_owner: "ondewo"
  comments: "trained on public domain dataset Lj_speech"
  speaker_name: "Linda"
  domain: "general"
}
active: true
inference {
  type: "composite"
  composite_inference {
    text2mel {
      type: "glow_tts"
      glow_tts {
        batch_size: 5
        length_scale: 2.0
        noise_scale: 0.6669999957084656
        path: "models/glow-tts/en/pretrained.pth"
        param_config_path: "models/glow-tts/en/config.json"
      }
      glow_tts_triton {
        batch_size: 8
        length_scale: 1.0
        noise_scale: 0.6669999957084656
        max_text_length: 100
        param_config_path: "models/glow-tts/en/config.json"
        triton_url: "localhost:50511"
        triton_model_name: "glow_tts"
      }
    }
    mel2audio {
      type: "hifi_gan"
      mb_melgan_triton {
        config_path: "models/mb_melgan/en/config.yml"
        stats_p

### Update pipeline

In [18]:
r = text_to_speech_pb2.ListT2sPipelinesRequest(speaker_sexes=0)
l = stab.ListT2sPipelines(request=r)
print(l)

pipelines {
  id: "glow_tts&hifi_gan-e976dd6c-2f41-484b-aec2-3e6868d37290"
  description {
    language: "de"
    speaker_sex: "female"
    pipeline_owner: "ondewo"
    comments: "trained on public domain dataset"
    speaker_name: "Kerstin"
    domain: "general"
  }
  active: true
  inference {
    type: "composite"
    composite_inference {
      text2mel {
        type: "glow_tts"
        glow_tts {
          batch_size: 5
          length_scale: 1.0
          noise_scale: 0.6669999957084656
          path: "models/glow-tts/de/kerstin_blank.pth"
          param_config_path: "models/glow-tts/de/config_blank.json"
        }
        glow_tts_triton {
          batch_size: 8
          length_scale: 1.0
          noise_scale: 0.6669999957084656
          max_text_length: 100
          param_config_path: "models/glow-tts/de/config_blank.json"
          triton_url: "localhost:50511"
          triton_model_name: "glow_tts"
        }
      }
      mel2audio {
        type: "hifi_gan"
       

### See if generated audio change according to updated config

In [19]:
stab.UpdateT2sPipeline(request=pipeline_config)

In [20]:
config = text_to_speech_pb2.RequestConfig(t2s_pipeline_id=english_pipeline.id)
request = text_to_speech_pb2.SynthesizeRequest(text='Hi, how are you?', config=config)
response = stab.Synthesize(request=request)

In [21]:
config = text_to_speech_pb2.RequestConfig(t2s_pipeline_id=english_pipeline.id)
request = text_to_speech_pb2.SynthesizeRequest(text='Hi, how are you?', config=config)
response = stab.Synthesize(request=request)

print(f'Length of the generated audio is {response.audio_length} sec.', f'Generation time is {response.generation_time} sec.')

bio = io.BytesIO(response.audio)

audio = sf.read(bio)

ipd.Audio(audio[0], rate=audio[1])

Length of the generated audio is 1.9620862007141113 sec. Generation time is 0.8621386289596558 sec.


### Change parameter back to previous (length_scale = 1.0)

In [22]:
request = text_to_speech_pb2.T2sPipelineId(id=english_pipeline.id)
pipeline_config = stab.GetT2sPipeline(request=request)
pipeline_config.inference.composite_inference.text2mel.glow_tts.length_scale = 1.0
stab.UpdateT2sPipeline(request=pipeline_config)

In [23]:
l = ['f', 'k']
from uuid import UUID

In [24]:
abs(hash(str(l)))

7271029856146044068

In [25]:
import torch


In [26]:
torch.__version__


'1.7.0'

In [27]:
config = text_to_speech_pb2.RequestConfig(t2s_pipeline_id=english_pipeline.id)
request = text_to_speech_pb2.BatchSynthesizeRequest(text=['Hola','Chau'], config=[config])
response = stab.BatchSynthesize(request=request)

In [28]:
response

response {
  audio_uuid: "7b0ef6e5-b118-4c8b-9d0c-b97405260be8"
  audio: "RIFF$N\000\000WAVEfmt \020\000\000\000\001\000\001\000\"V\000\000D\254\000\000\002\000\020\000data\000N\000\000\365\377\032\000\024\000\007\000\n\000\000\000\014\000\357\377\376\377\001\000\377\377\352\377\374\377\371\377\355\377\362\377\375\377\375\377\362\377\364\377\005\000\375\377\367\377\372\377\006\000\007\000\001\000\373\377\010\000\r\000\374\377\355\377\375\377\365\377\362\377\375\377\005\000\001\000\361\377\366\377\n\000\000\000\377\377\364\377\004\000\016\000\372\377\010\000\023\000\004\000\374\377\013\000\014\000\002\000\006\000\377\377\376\377\001\000\371\377\345\377\354\377\354\377\366\377\347\377\366\377\370\377\363\377\372\377\366\377\007\000\n\000\374\377\365\377\027\000\036\000\371\377\000\000\027\000\021\000\370\377\025\000\033\000\004\000\001\000\374\377\026\000 \000\374\377\016\000 \000\016\000\003\000\017\000\025\000\025\000\021\000\025\000\373\377\353\377\347\377\000\000\000\000\364\377\007\